In [42]:
from wilcoxon.spiderman import *
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
import shapely
from wilcoxon.utils import *

In [12]:
def convertCoords(d,m,s):
    return float(d) + float(m)/60 + float(s)/3600

In [13]:
# MRT

stations = website("https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations")
stations.getTables()
stationLinksM = stations.tables[1]["Links"].apply(lambda x: x.split("\n")[0]).tolist()
stationLinksM = [x for x in stationLinksM if ("a"+x)[-1] == "n"]
stationLinksM.extend(["https://en.wikipedia.org/wiki/Changi_Airport_MRT_station",
                     "https://en.wikipedia.org/wiki/Expo_MRT_station",
                     "https://en.wikipedia.org/wiki/Tanah_Merah_MRT_station"])

In [14]:
# LRT

stations = website("https://en.wikipedia.org/wiki/List_of_Singapore_LRT_stations")
stations.getTables()
stationLinksL = stations.tables[0]["Links"].apply(lambda x: x.split("\n")[0]).tolist()
stationLinksL = [x for x in stationLinksL if ("a"+x)[-1] == "n"]
stationLinksL.extend(["https://en.wikipedia.org/wiki/Damai_LRT_station_(Singapore)"])

In [15]:
# COMBINED

stationLinks = list(set(stationLinksM + stationLinksL))

In [16]:
def getMRTFromLink(link):
    site = website(link)
    try:
        stationName = site.html("title")[0].string.split(" MRT")[0]
    except Exception as e:
        print("stationName gave the error: " + str(e) + "\n" + link)
        stationName = None
    
    try:
        # stationLabels = ", ".join(list(site.html(class_="nobold")[0].stripped_strings))
        
        # Station name in different languages, and label
        stationDetails = list(site.html.find(class_="fn org").stripped_strings)
        chineseIndex = 0
        for detail in stationDetails:
            if re.compile('[\u4e00-\u9fff]+').search(detail):
                chineseIndex = stationDetails.index(detail)
        stationLabels = ", ".join(stationDetails[:chineseIndex-1])
        stationChinese = stationDetails[chineseIndex]
    except Exception as e:
        print("stationLabels gave the error: " + str(e) + "\n" + link)
        stationLabels = None
        stationChinese = None
    
    try:
        lat = convertCoords(*re.findall("[0-9.]+", site.html(class_="latitude")[0].text))
        long = convertCoords(*re.findall("[0-9.]+", site.html(class_="longitude")[0].text))
    except Exception as e:
        print("latlong gave the error: " + str(e) + "\n" + link)
        lat = None
        long = None

    try:
        fullAddress = list(site.html(class_="infobox-data")[0].strings)
        address = fullAddress[0]
        postcode = re.findall(r"\d{6}", fullAddress[1])[0]
    except Exception as e:
        print("address gave the error: " + str(e) + "\n" + link)
        address = None
        postcode = None
    
    return {"Label": stationLabels, "Name": stationName, "Chinese": stationChinese, "Address": address, "Postcode": postcode, "Lat": lat, "Long": long, "Link": link}

In [17]:
mrt = []

for link in tqdm(stationLinks):
    mrt.append(getMRTFromLink(link))
    
mrt = pd.DataFrame(mrt)

  4%|▎         | 8/224 [00:13<06:01,  1.67s/it]

address gave the error: list index out of range
http://wikipedia.org/wiki/Brickland_MRT_station


  9%|▉         | 20/224 [00:31<04:24,  1.30s/it]

latlong gave the error: list index out of range
http://wikipedia.org/wiki/Sungei_Kadut_MRT_station
address gave the error: list index out of range
http://wikipedia.org/wiki/Sungei_Kadut_MRT_station


 11%|█         | 25/224 [00:40<05:19,  1.61s/it]

...

 98%|█████████▊| 219/224 [05:31<00:06,  1.23s/it]

address gave the error: list index out of range
http://wikipedia.org/wiki/Peng_Kang_Hill_MRT_station


100%|█████████▉| 223/224 [05:36<00:01,  1.40s/it]

address gave the error: list index out of range
http://wikipedia.org/wiki/Mount_Pleasant_MRT_station


100%|██████████| 224/224 [05:38<00:00,  1.51s/it]


In [56]:
## Abbreviations

site = website("https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations")
abbrMapping = {}
for t in site.html("td",text=re.compile("^[A-Z]{3}$")):
    if t.text != "TBA":
        abbrMapping[t.find_previous("a",href=True).text.strip()] = t.text
abbrMapping

def tryAbb(x):
    try: return abbrMapping[x]
    except: return ""

mrt["Abbreviation"] = mrt.Name.apply(tryAbb)

In [57]:
mrt.Abbreviation.to_clipboard(index=False)

In [481]:
exits = gpd.read_file("Documents/Geospatial/GEOSPATIAL/TrainStationExit_Jan2020/TrainStationExit06032020.shp")
exits["STN_NAME"] = exits["STN_NAME"].str.replace(" .RT STATION","").str.replace(" STATION","")
exits = exits[["STN_NAME","EXIT_CODE","geometry"]]
exits.columns = ["Name","Exit","geometry"]

stations = gpd.read_file("Documents/Geospatial/GEOSPATIAL/TrainStation_Jan2020/MRTLRTStnPtt.shp")
stations["geometry"] = stations.geometry.to_crs(epsg=4326)
stations["STN_NAME"] = stations["STN_NAME"].str.replace(" .RT STATION","").str.replace(" STATION","")
stations = stations[["STN_NAME","STN_NO","geometry"]]
stations.columns = ["Name","Exit","geometry"]

mrtFull = mrt.copy()
mrtFull["Name"] = mrtFull.Name.str.upper().str.replace(" .RT STATION.*$","")
mrtFull = pd.concat([mrtFull.merge(stations, how="left"),
                     mrtFull.merge(exits, how="left")])

mrtFull["Exit"] = mrtFull[["Exit","Label"]].apply(lambda x: x[1] if pd.isna(x[0]) else x[0], axis=1)
mrtFull = gpd.GeoDataFrame(mrtFull, geometry="geometry")
mrtFull["Labels"] = mrtFull.Label.copy()
mrtFull["Label"] = mrtFull["Exit"]
mrtFull = mrtFull.drop("Exit",axis=1)

mrtFull = mrtFull[["Label","Name","Chinese","Address","Postcode","Lat","Long","Labels","geometry","Link"]]
mrtFull["Name"] = mrtFull.Name.str.replace(" LRT.*?$","")
mrtFull["Long"] = mrtFull[["geometry","Long"]].apply(lambda x: x[1] if pd.isna(x[0]) else x[0].x, axis=1)
mrtFull["Lat"] = mrtFull[["geometry","Lat"]].apply(lambda x: x[1] if pd.isna(x[0]) else x[0].y, axis=1)

mrtFull["Label"] = mrtFull.Label.str.split("[,/]")
mrtFull = gpd.GeoDataFrame(pd.DataFrame(mrtFull).explode("Label"))
mrtFull["Label"] = mrtFull.Label.str.strip()
mrtFull["line"] = mrtFull.Label.str.extract("(\D+)").fillna(0)
mrtFull = mrtFull[mrtFull.line != 0]

passengers = pd.read_csv("Documents/Geospatial/GEOSPATIAL/origin_destination_train_202103.csv")
passengers.ORIGIN_PT_CODE = passengers.ORIGIN_PT_CODE.str.split("/")
passengers.DESTINATION_PT_CODE = passengers.DESTINATION_PT_CODE.str.split("/")
passengers = passengers.explode("ORIGIN_PT_CODE")
passengers = passengers.explode("DESTINATION_PT_CODE")
passengers.ORIGIN_PT_CODE = passengers.ORIGIN_PT_CODE.str.strip()
passengers.DESTINATION_PT_CODE = passengers.DESTINATION_PT_CODE.str.strip()

mrtFull = mrtFull.merge(pd.DataFrame(passengers.groupby("ORIGIN_PT_CODE").pipe(lambda x: x.TOTAL_TRIPS.sum())),
                        left_on="Label", right_on="ORIGIN_PT_CODE", how="left")
mrtFull = mrtFull.merge(pd.DataFrame(passengers.groupby("DESTINATION_PT_CODE").pipe(lambda x: x.TOTAL_TRIPS.sum())),
                        left_on="Label", right_on="DESTINATION_PT_CODE", how="left")
mrtFull = mrtFull.rename(columns={"TOTAL_TRIPS_x":"Origin", "TOTAL_TRIPS_y":"Destination"})

colorMap = {"EW":"#009645",
           "CG":"#009645",
           "NS":"#D42E12",
           "NE":"#9900AA",
           "CC":"#FA9E0D",
           "CE":"#FA9E0D",
           "DT":"#005EC4",
           "TE":"#9D5B25",
           "JE":"#0099AA",
           "JS":"#0099AA",
           "JW":"#0099AA",
           "CR":"#97C616",
           "CP":"#97C616",
           "PE":"#748477",
           "PW":"#748477",
           "PT":"#748477",
           "ST":"#748477",
           "SE":"#748477",
           "SW":"#748477",
           "BP":"#748477",
           "RT":"#86CEEB"}

mrtFull["color"] = mrtFull.Label.apply(lambda x: colorMap[x[:2]] if x[:2] in colorMap.keys() else None)
mrtFull["number"] = mrtFull.Label.str.extract("(\d+)").fillna("1000")
mrtFull["number"] = mrtFull.number.apply(int)
mrtFull["Type"] = mrtFull.line.apply(lambda x: "Station" if len(x) > 1 else "Exit")

mrtFull = mrtFull[['Label', 'Name', 'Chinese', 'Address', 'Postcode', 'Lat', 'Long', 
                   'Labels', 'Type', 'Origin', 'Destination', 'geometry', 'color', 'line', 'number', 'Link']]

In [483]:
mrtFull.sort_values(["Type","line","number"], ascending=False).drop(["line","number"], axis=1).to_clipboard(index=False)

In [1]:
from wilcoxon.geo import *

In [2]:
mrt = getMRTFromWiki()

Extracting Wikipedia links of MRT and LRT stations.


  0%|          | 0/4 [00:00<?, ?it/s]

Extracting data from Wikipedia pages.


100%|██████████| 4/4 [00:02<00:00,  1.90it/s]


Mapping station names to their abbreviations.


AttributeError: 'list' object has no attribute 'Name'

In [6]:
stations = website("https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations")
stations.getTables()
stationLinksM = stations.tables[1]["Links"].apply(lambda x: x.split("\n")[0]).tolist()
stationLinksM = [x for x in stationLinksM if ("a"+x)[-1] == "n"]
stationLinksM.extend(["https://en.wikipedia.org/wiki/Changi_Airport_MRT_station",
                     "https://en.wikipedia.org/wiki/Expo_MRT_station",
                     "https://en.wikipedia.org/wiki/Tanah_Merah_MRT_station"])

stations = website("https://en.wikipedia.org/wiki/List_of_Singapore_LRT_stations")
stations.getTables()
stationLinksL = stations.tables[0]["Links"].apply(lambda x: x.split("\n")[0]).tolist()
stationLinksL = [x for x in stationLinksL if ("a"+x)[-1] == "n"]
stationLinksL.extend(["https://en.wikipedia.org/wiki/Damai_LRT_station_(Singapore)"])

stationLinks = list(set(stationLinksM + stationLinksL))

In [12]:
stationLinksL

['https://en.wikipedia.org/wiki/Damai_LRT_station_(Singapore)']